In [1]:
import tensorflow as tf
import numpy as np

In [2]:
##### A simple RNN network of 6 neurons without the power of tf cells
n_inputs = 3
n_neurons = 6

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()


In [3]:
#                  instance0, instance1, instance2, instance3
X0_batch = np.array([[0,1,2],   [3,4,5],   [6,7,8],  [9,0,1]]) # t=0
X1_batch = np.array([[9,8,7],   [0,0,0],   [6,5,4],  [3,2,1]]) # t=1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})


In [4]:
print(Y0_val)

[[ 0.76432037 -0.91695583  0.97727591 -0.32062194 -0.67042279  0.14208993]
 [ 0.07333915 -0.99902272  1.         -0.99996823 -0.99998587 -0.99986333]
 [-0.6960423  -0.99998891  1.         -1.         -1.         -1.        ]
 [-0.66256243  0.99172556  1.         -0.99999994 -0.99999642 -1.        ]]


In [5]:
print(Y1_val)

[[-0.9996497  -0.99996477  1.         -1.         -1.         -1.        ]
 [-0.41848999  0.67247814 -0.23164959  0.9981429  -0.63004768  0.97460502]
 [-0.99871987 -0.5623796   1.         -0.9999975  -1.         -0.99999976]
 [-0.64565659 -0.90720129  0.99999148 -0.46441022 -0.99998224 -0.9999308 ]]


In [6]:
sess.close()
tf.reset_default_graph()

In [7]:
##### Now the same RNN network with the use of tf cells

X_0 = tf.placeholder(tf.float32, [None, n_inputs])
X_1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X_0, X_1], dtype=tf.float32)
Y_0, Y_1 = output_seqs

In [8]:
init = tf.global_variables_initializer()
#                  instance0, instance1, instance2, instance3
X_0_batch = np.array([[0,1,2],   [3,4,5],   [6,7,8],  [9,0,1]]) # t=0
X_1_batch = np.array([[9,8,7],   [0,0,0],   [6,5,4],  [3,2,1]]) # t=1

with tf.Session() as sess:
    init.run()
    Y_0_val, Y_1_val = sess.run([Y_0, Y_1], feed_dict={X_0: X_0_batch, X_1: X_1_batch})

print(Y_0_val)
print(Y_1_val)

[[-0.50482887  0.80509692  0.45999557  0.03189753  0.93911433 -0.79291445]
 [-0.99517113  0.99324089  0.79699486 -0.50808209  0.99994242 -0.99991232]
 [-0.99996436  0.99978703  0.93329275 -0.81847847  0.99999982 -0.99999982]
 [-0.99997419 -0.93404073 -0.98145795 -0.74395895  0.42677706 -0.99974197]]
[[ -9.99999642e-01   9.99612868e-01   8.24952900e-01  -9.39361215e-01
    9.99999940e-01  -1.00000000e+00]
 [ -9.20558631e-01   5.66656768e-01  -4.47062820e-01  -3.34600717e-01
    5.60612857e-01  -2.99190283e-01]
 [ -9.99991357e-01   9.96022046e-01   3.16824056e-02  -9.36235011e-01
    9.99991059e-01  -9.99999166e-01]
 [ -9.86770630e-01   7.47967899e-01   2.34305830e-04  -7.64231741e-01
    9.79189038e-01  -9.56827343e-01]]


In [9]:
sess.close()
tf.reset_default_graph()

In [10]:
##### Now the same RNN networks with tf cells and just a tensor with our n_steps
n_inputs = 3
n_neurons = 6
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2])) # we transforme X in a list of n_steps tensors of shape[None, n_inputs]
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)

outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [11]:
X_batch = np.array([
    # t = 0    t = 1
    [[0,1,2], [9,8,7]], # instance 0
    [[3,4,5], [0,0,0]], # instance 1
    [[6,7,8], [6,5,4]], # instance 2
    [[9,0,1], [3,2,1]], # instance 3
])

In [12]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [13]:
print(outputs_val)

[[[ 0.74332833 -0.29883853  0.00570595 -0.49935222 -0.28266123 -0.31899813]
  [ 0.95952499 -0.08961026  0.99999583  0.95828569  0.99915183 -0.99999833]]

 [[ 0.95158476 -0.37783515  0.9595896  -0.01568618  0.71562892 -0.98797143]
  [-0.33797461 -0.83774149  0.14631489  0.31666163  0.40735844 -0.87474936]]

 [[ 0.9916746  -0.45168886  0.99913996  0.47543386  0.96973479 -0.99985832]
  [ 0.3373484  -0.90567183  0.99912858  0.97427893  0.99627727 -0.99996912]]

 [[-0.99209398 -0.73277915  0.99997538  0.62479281  0.9999373  -0.99960953]
  [-0.09267516 -0.67475277  0.88905203  0.94988328  0.67071736 -0.98185629]]]


In [14]:
sess.close()
tf.reset_default_graph()

In [16]:
##### Now we use Dynamic Unrolling that permit us to run over the cell the appropriate number of times
##### we can also set swap_memory=True if we want to swap the GPU's memory to the CPU's memory during 
##### backpropagation to avoid Out of memory errors.
n_inputs = 3
n_neurons = 6
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [17]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [18]:
print(outputs_val)

[[[-0.53025371  0.58454984 -0.60889852 -0.75752777  0.90663368  0.40981999]
  [-0.99605227 -0.9983173  -0.51188105 -0.87043715  1.          0.99765992]]

 [[-0.98093081 -0.33391705 -0.8171016  -0.92095989  0.99998981  0.93225205]
  [ 0.90081394  0.18810864 -0.28867733 -0.38355374 -0.04714653 -0.00770936]]

 [[-0.99939644 -0.87727314 -0.91998017 -0.9757545   1.          0.99414402]
  [-0.92071438 -0.98946625 -0.57992405 -0.51652431  0.99999958  0.95227605]]

 [[-0.99730086 -0.99990761  0.99691075  0.8216545   0.99995518  0.95956993]
  [-0.86190498 -0.98064798  0.84106147  0.70973712  0.96618319  0.53029609]]]


In [27]:
sess.close()
tf.reset_default_graph()

In [28]:
##### Now we starting to add complexity. What happend when the input sequence have variable lenghts
##### such in words and sentences? We have to add the lenght of our sequence inputs 

seq_length = tf.placeholder(tf.float32, [None]) # the lenght of our input value

##### Suppose one of our input sequences contains only one input instead of two, like in instance 1

X_batch = np.array([
    # t = 0    t = 1
    [[0,1,2], [9,8,7]], # instance 0
    [[3,4,5]], # instance 1
    [[6,7,8], [6,5,4]], # instance 2
    [[9,0,1], [3,2,1]], # instance 3
])

##### then we have to add a padding of zeros

X_batch = np.array([
    # t = 0    t = 1
    [[0,1,2], [9,8,7]], # instance 0
    [[3,4,5], [0,0,0]], # instance 1
    [[6,7,8], [6,5,4]], # instance 2
    [[9,0,1], [3,2,1]], # instance 3
])
seq_length_batch = np.array([2,1,2,2])

In [29]:
##### ...and then our dynamic RNN become
n_inputs = 3
n_neurons = 6
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length) # we add the lenght values

In [30]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run(
        [outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In [31]:
print(outputs_val)

[[[-0.87804133  0.30256867  0.83663434 -0.08793465 -0.63203824  0.61946684]
  [-0.9999423   0.72810918  0.99999636 -0.43976662 -0.99994302  0.99977416]]

 [[-0.99786919  0.27504775  0.99860525 -0.3433398  -0.99650282  0.98064446]
  [ 0.          0.          0.          0.          0.          0.        ]]

 [[-0.99996501  0.24706851  0.99998915 -0.55638719 -0.99997288  0.99918735]
  [-0.99812543  0.94318289  0.99978596 -0.37332469 -0.97179782  0.99302483]]

 [[ 0.67462504  0.22138624 -0.44368061 -0.95958883 -0.99317026  0.98140597]
  [-0.72439784  0.43750948  0.98660934 -0.68022025 -0.64432818  0.2375631 ]]]


In [32]:
print(states_val)

[[-0.9999423   0.72810918  0.99999636 -0.43976662 -0.99994302  0.99977416]
 [-0.99786919  0.27504775  0.99860525 -0.3433398  -0.99650282  0.98064446]
 [-0.99812543  0.94318289  0.99978596 -0.37332469 -0.97179782  0.99302483]
 [-0.72439784  0.43750948  0.98660934 -0.68022025 -0.64432818  0.2375631 ]]
